In [1]:
import requests
from urllib.parse import urlparse #한글처리
import urllib.parse
from tqdm import tqdm
import time
from urllib.request import urlopen
from bs4 import BeautifulSoup
import requests
import pandas as pd
import logging
import traceback

In [ ]:
df = pd.read_csv('파일이름', encoding='utf-8')

In [ ]:
df

In [48]:
url = 'https://movie.naver.com/movie/bi/mi/basic.naver?code=201641'

In [61]:
# 관람객 평점
aud_score=soup.findAll("span",{"class":"st_on"})[0].text.split(" ")[2].replace('점',"")
aud_score

'8.03'

In [ ]:
aud_score=[]
for i in tqdm(range(0,100)):
    try :
        # 페이지 요청
        html = requests.get(url).content
        # 파싱객체 생성
        soup = BeautifulSoup(html,"html.parser")
        time.sleep(1)
        aud=soup.findAll("span",{"class":"st_on"})[0].text.split(" ")[2].replace('점',"")
        aud_score.append(aud)
    except Exception as e:        
        logging.error(traceback.format_exc())
        score.append('-')

In [50]:
# 전문가 평점
exp_score=soup.findAll("div",{"class":"spc_score_area"})[0].text.split("\n\n")[2]
exp_score

'5.50'

In [ ]:
spc_score=[]
for i in tqdm(range(0,100)):
    try :
        # 페이지 요청
        html = requests.get(url).content
        # 파싱객체 생성
        soup = BeautifulSoup(html,"html.parser")
        time.sleep(1)
        spc=soup.findAll("div",{"class":"spc_score_area"})[0].text.split("\n\n")[2]
        spc_score.append(spc)
    except Exception as e:        
        logging.error(traceback.format_exc())
        score.append('-')

In [59]:
# 네티즌 평점
# star_score 
ntz_score=soup.findAll("a",{"id":"pointNetizenPersentBasic"})[0].text
ntz_score

'8.62'

In [ ]:
ntz_score=[]
for i in tqdm(range(0,100)):
    try :
        # 페이지 요청
        html = requests.get(url).content
        # 파싱객체 생성
        soup = BeautifulSoup(html,"html.parser")
        time.sleep(1)
        ntz=soup.findAll("a",{"id":"pointNetizenPersentBasic"})[0].text
        ntz_score.append(ntz)
    except Exception as e:        
        logging.error(traceback.format_exc())
        score.append('-')

HDFS 저장

In [ ]:
file_dir = '/movie_data/score/'
file_name = 'movie_score_' + cal_std_day(1) + '.json'

url = 'https://ncv.kdca.go.kr/mainStatus.es?mid=a11702000000'
response_txt = execute_rest_api('get',url,{},{})
soup = BeautifulSoup(response_txt, 'html.parser')
trs = soup.select('#content > div.data_table.tbl_scrl_t > table > tbody > tr')

cols = ['loc', 'v1', 'v2', 'v3', 'v4']
data = []

for idx, tr in enumerate(trs):
    if idx == 0:
        continue
    th = tr.select('th')
    tds = tr.select('td')
    
    rows = []
    rows.append(th[0].text.replace(' ', '').replace('\r\n', ''))
    
    for idx, td in enumerate(tds):
        if idx % 2 == 0:
            continue            
        rows.append(td.text.replace(' ', '').replace('\r\n', '').replace(',' , ''))
    
    tmp = dict(zip(cols, rows))
    data.append(tmp)

res = {
    'meta':{
        'desc':'지역별 코로나 예방접종 인구 현황',
        'cols':{
            'loc':'지역'
            ,'v1':'1차접종자수'
            ,'v2':'2차접종자수'
            ,'v3':'3차접종자수'
            ,'v4':'4차접종자수'
        },
        'std_day':cal_std_day(1)
    },
   'data':data
}

client.write(file_dir+file_name, json.dumps(res, ensure_ascii=False), encoding='utf-8')